In [1]:
%%time
import glob
import pandas as pd
import time

Wall time: 482 ms


In [2]:
%%time
DF = pd.DataFrame(columns=["fileName", "text"])
st_time = time.time()
paths = glob.glob(r'C:\Users\QuadAdmin\Downloads\aclImdb\**\*.txt',recursive = True)
# print(f"time took: {(time.time() - st_time):.2f} seconds")

Wall time: 933 ms


In [3]:
DF["fileName"] = paths

In [4]:
DF

,fileName,text
0,C:\Users\QuadAdmin\Downloads\aclImdb\all\test\...,NaN
1,C:\Users\QuadAdmin\Downloads\aclImdb\all\test\...,NaN
2,C:\Users\QuadAdmin\Downloads\aclImdb\all\test\...,NaN
3,C:\Users\QuadAdmin\Downloads\aclImdb\all\test\...,NaN
4,C:\Users\QuadAdmin\Downloads\aclImdb\all\test\...,NaN
...,...,...
99995,C:\Users\QuadAdmin\Downloads\aclImdb\all - Cop...,NaN
99996,C:\Users\QuadAdmin\Downloads\aclImdb\all - Cop...,NaN
99997,C:\Users\QuadAdmin\Downloads\aclImdb\all - Cop...,NaN
99998,C:\Users\QuadAdmin\Downloads\aclImdb\all - Cop...,NaN


In [5]:
import findspark
findspark.init()

In [6]:
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

In [7]:
%%time
conf = SparkConf()
conf.set("spark.network.timeout", "1000000")
conf.set("spark.executor.heartbeatInterval", "1000000")
conf.set("spark.speculation","false")
conf.set("spark.app.name","blaher")
spark = SparkSession.builder.config(conf = conf).master("local[*]").enableHiveSupport().getOrCreate() #("local[*]") uses all the cpu cores#

Wall time: 6.14 s


In [8]:
spark

In [9]:
sc = spark.sparkContext

In [10]:
# ls_ran = list(range(1,41))
# len(ls_ran)

In [11]:
# ls_rdd = sc.parallelize(ls_ran)
# ls_rdd.take(5)

In [12]:
# type(ls_rdd)

In [13]:
# ls_rdd.getNumPartitions()

In [14]:
def readfiles(path):
    with open(path,"r",encoding="utf-8") as file:
        text = file.read()
    return text

In [15]:
DF["fileName"][0]

'C:\\Users\\QuadAdmin\\Downloads\\aclImdb\\all\\test\\neg\\0_2.txt'

In [16]:
# readfiles(DF["fileName"][0])
with open(DF["fileName"][0], "r", encoding="utf-8") as file:
    print(file.read())

Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he's better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutcher's ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hour in.


In [17]:
paths_rdd = sc.parallelize(paths)

In [18]:
text_rdd = paths_rdd.map(readfiles)

In [19]:
%%time
DF["text"] = DF["fileName"].apply(lambda path: readfiles(path))

KeyboardInterrupt: 

In [ ]:
DF

In [ ]:
from pyspark.sql.types import *

In [ ]:
from pyspark.sql import *

In [ ]:
import pyspark.sql.functions as F

In [ ]:
from parse_pii.piiparser import *

In [ ]:
DF["text"][0]

In [ ]:
classify(DF["text"][0])

In [ ]:
sparkDF = spark.createDataFrame(DF)

In [ ]:
sparkDF.show(5)

In [ ]:
sparkDF.rdd.getNumPartitions()

In [ ]:
classify_udf = F.udf(classify, StringType())

In [ ]:
# start_time = time.time()
# sparkDF.select("fileName", classify_udf("text").alias("has_pii")).show(5)
# print(f"time took: {(time.time() - start_time):.2f} seconds")

In [ ]:
# fDF = sparkDF.select("fileName", classify_udf("text").alias("has_pii"))

In [ ]:
# start_time = time.time()
# fDF.coalesce(2).write.mode("overwrite").format("csv").option("header","true").save("output")
# print(f"time took: {(time.time() - start_time):.2f} seconds")

In [ ]:
# fDF.printSchema()

In [ ]:
import spacy
from presidio_analyzer import AnalyzerEngine
from presidio_analyzer.nlp_engine import SpacyNlpEngine
from presidio_analyzer.recognizer_registry import RecognizerRegistry

In [ ]:
def entity_counter(entity_list):
    freq = {}
    entity_list = sorted(entity_list)
    for items in entity_list: 
        freq[items] = entity_list.count(items) 
 
    counter = ""
    for key, value in freq.items(): 
        if key == entity_list[-1]:
            counter += f"{key}:{value}"
        else:
            counter += f"{key}:{value}"+", "
    return counter

In [ ]:
analyzer = AnalyzerEngine(nlp_engine=nlp)

In [ ]:
def pii_classify(stext):
    text = stext
    response = analyzer.analyze(correlation_id=0, text = text,language="en",entities=[], score_threshold=0.5)
    if len(response) == 0:
        return "no"
    else:
        return "yes"

In [ ]:
(DF["text"][1000])

In [ ]:
pii_classify(DF["text"][1000])

In [ ]:
pii_apply_udf = F.udf(pii_classify, StringType())

In [ ]:
%%time
sparkDF.select("fileName", classify_udf("text").alias("has_pii")).show(5)

In [ ]:
%%time
fDF = sparkDF.select("fileName", classify_udf("text").alias("has_pii"))
fDF = fDF.repartition(8)
fDF.show(5)

In [ ]:
# fDF.write.mode("overwrite").csv("op_table2")

In [ ]:
# fDF.repartition(1).write.mode("overwrite").csv("output",header=True)

In [ ]:
# start_time = time.time()
# sparkDF.coalesce(2).write.mode("overwrite").format("csv").option("header","true").save("output")
# print(f"time took: {(time.time() - start_time):.2f} seconds")